In [10]:
import tensorflow as tf
import pandas as pd
from transformers import AutoTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
base_df = pd.read_csv("home_appliances.csv").dropna()
base_df.head(2)

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,5215,일주일째 쓰고 있는데 진동은 강한데 이물질은 잘 빠지지 않아서 그닥. 칫솔모도 너무...,SNS,가전,생활/미용/욕실가전,(방송중2만원할인) 필립스 프로텍티브 클린 1.0 더블핸들 (HX6456/03),70,61,15,20200514,0.0,기능,진동은 강한데 이물질은 잘 빠지지 않아서 그닥,7,0
1,5215,일주일째 쓰고 있는데 진동은 강한데 이물질은 잘 빠지지 않아서 그닥. 칫솔모도 너무...,SNS,가전,생활/미용/욕실가전,(방송중2만원할인) 필립스 프로텍티브 클린 1.0 더블핸들 (HX6456/03),70,61,15,20200514,0.0,사이즈,칫솔모도 너무 작아서 저한테는 별로예요,5,-1


In [21]:
label_encoder = LabelEncoder()
enc_data = label_encoder.fit_transform(base_df["Aspect"])
num_labels = len(set(enc_data))

In [22]:
HUGGING_FACE_PATH = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(HUGGING_FACE_PATH, num_labels=num_labels, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_PATH)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
X, y = base_df.loc[:, "RawText"].to_list(), enc_data

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=88)

In [25]:
X_train_encoding = tokenizer(X_train, padding=True, truncation=True, max_length=42)

In [26]:
SHUFFLE_PARAM = 1000

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoding),
    y_train
)).shuffle(SHUFFLE_PARAM)

In [27]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, metrics=["accuracy"])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  15380     
                                                                 
Total params: 110,632,724
Trainable params: 110,632,724
Non-trainable params: 0
_________________________________________________________________


In [28]:
BATCH_PARAM = 32

validation_length = len(X_train) // 10
train_except_val = train_dataset.skip(validation_length).batch(BATCH_PARAM)
validation_data = train_dataset.take(validation_length).batch(BATCH_PARAM)

In [ ]:
model.fit(
    train_except_val,
    epochs=1,
    batch_size=BATCH_PARAM,
    validation_data=validation_data)

  16/2026 [..............................] - ETA: 10:52:48 - loss: 2.5964 - accuracy: 0.2637